## Ejemplo predicción de precios.

La base de datos se puede encontrar en:<p>
https://www.kaggle.com/harlfoxem/housesalesprediction/data<p>
Predicción del precio de casas en King County, USA.<p>
Leo los datos y observo si hay correlación entre la variable que quiero predecir (precio) y el resto:<p>

In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('kc_house_data.csv')
data.corr().ix[:, 1]

id              -0.016762
price            1.000000
bedrooms         0.308350
bathrooms        0.525138
sqft_living      0.702035
sqft_lot         0.089661
floors           0.256794
waterfront       0.266369
view             0.397293
condition        0.036362
grade            0.667434
sqft_above       0.605567
sqft_basement    0.323816
yr_built         0.054012
yr_renovated     0.126434
zipcode         -0.053203
lat              0.307003
long             0.021626
sqft_living15    0.585379
sqft_lot15       0.082447
Name: price, dtype: float64

Realizo un tratamiento de datos, transformo en variables categóricas (zipcode, grade, waterfront y condition) y posteriormente genero variables dummy. Elimino (id, date y long)

In [2]:
data["zipcode"] = data["zipcode"].astype('category')
data["grade"] = data["grade"].astype('category')
data["waterfront"] = data["waterfront"].astype('category')
data["condition"] = data["condition"].astype('category')
data.drop(["id", "date", "long"], axis=1, inplace = True)
data = pd.get_dummies(data)

Divido la muestra en entrenamiento, validación y test.(60/20/20)

In [3]:
from sklearn import preprocessing
train, val, test = np.split(data.sample(frac=1, random_state=1),
                            [int(.6*len(data)), int(.8*len(data))])
y_train = train["price"]
y_val = val["price"]
y_test = test["price"]
X_train = train.ix[:, 1:103]
X_val = val.ix[:, 1:103]
X_test = test.ix[:, 1:103]

Calculo distintos modelos en función de $C$ y $d$ <p>
En scikit-learn es $\alpha=1/C$ y el grado del polinomio (d). <p>
Calculo el error cuadrático medio para los tres data sets.

In [4]:
from sklearn import metrics, linear_model

list_C = list([0.1, 0.0001,0.00000001,  0.000000001, 0.000000005, 0.0000000001]) #0.01 mejor
list_d = range(1,3)
mse_train = np.zeros(len(list_C)*len(list_d))
mse_val = np.zeros(len(list_C)*len(list_d))
mse_test = np.zeros(len(list_C)*len(list_d))
degree = np.zeros(len(list_C)*len(list_d))
param_C = np.zeros(len(list_C)*len(list_d))
R_test = np.zeros(len(list_C)*len(list_d))
count = 0
for d in list_d:
    MapFeature = preprocessing.PolynomialFeatures(degree=d)
    for C in list_C:
        reg = linear_model.Ridge(alpha = 1/C)
        reg.fit(MapFeature.fit_transform(X_train), y_train)
        mse_train[count] = metrics.mean_squared_error(y_train,
                            reg.predict(MapFeature.fit_transform(X_train)))
        mse_val[count] = metrics.mean_squared_error(y_val,
                          reg.predict(MapFeature.fit_transform(X_val)))
        mse_test[count] = metrics.mean_squared_error(y_test, 
                           reg.predict(MapFeature.fit_transform(X_test)))
        R_test[count] = reg.score(MapFeature.fit_transform(X_test), 
                                  y_test)
        degree[count] = d
        param_C[count]  = C
        count = count + 1

Observo los distintos modelos y su error cuadratico medio (ECM).Es decir:<p>
$J_\text{train}(\theta)=$ Train ECM <p>
$J_\text{val}(\theta)=$ Validation ECM <p>
$J_\text{test}(\theta)=$ Test ECM <p>

In [5]:
matrix = np.matrix(np.c_[degree, param_C, mse_train, 
                         mse_val, mse_test, R_test])
models = pd.DataFrame(data = matrix, columns = 
             ['degree d', 'parameter C', 'Train ECM', 
              'Validation ECM', 'Test ECM', 'R^2 Test'])
models.ix[:, :5].head(n=10)

,degree d,parameter C,Train ECM,Validation ECM,Test ECM
0,1.0,1.000000e-01,2.391792e+10,1.906537e+10,2.410939e+10
1,1.0,1.000000e-04,5.799330e+10,4.599485e+10,5.930629e+10
2,1.0,1.000000e-08,6.747692e+10,5.426893e+10,6.903229e+10
3,1.0,1.000000e-09,6.836345e+10,5.448866e+10,6.985207e+10
4,1.0,5.000000e-09,6.776303e+10,5.447129e+10,6.932688e+10
5,1.0,1.000000e-10,7.700754e+10,5.917042e+10,7.855262e+10
6,2.0,1.000000e-01,8.981384e+09,2.285930e+10,1.829025e+10
7,2.0,1.000000e-04,1.117378e+10,2.343748e+10,1.876143e+10
8,2.0,1.000000e-08,1.270795e+10,1.757209e+10,1.752027e+10
9,2.0,1.000000e-09,1.529776e+10,1.934655e+10,1.885666e+10


Elijo el modelo que menor error cuadrático medio tiene en el conjunto de validación.<p>
La capacidad de generalización vendrá dada por el error en el conjunto de test <p>
Esto es debido a que es totalmente independiente al modelo elegido.<p>
Si evaluasemos el error de generalización con el conjunto de validación <p>
tenderíamos a sobreestimarlo dado que ha sido un parámetro a la hora de elegir el modelo.

In [6]:
best_index = models['Validation ECM'].idxmin()
print('Modelo con menor ECM en el conjunto de validación')
models.ix[best_index, :]

Modelo con menor ECM en el conjunto de validación


degree d          2.000000e+00
parameter C       1.000000e-08
Train ECM         1.270795e+10
Validation ECM    1.757209e+10
Test ECM          1.752027e+10
R^2 Test          8.776310e-01
Name: 8, dtype: float64

Mido la capacidad de generalización por $J_\text{test}(\theta)$, dado que el test set es independiente a la elección del modelo.<p>
El coeficiente de determinacion $R^2$ es 0.878